<a href="https://colab.research.google.com/github/lunarcogito/synthpop_gan/blob/master/pretreatment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install pypianoroll

  Created wheel for pypianoroll: filename=pypianoroll-0.5.2-cp36-none-any.whl size=23489 sha256=0f8fd2fee4bf977c30c1c9dfd9e1ff06dcef154de6cac29de6475fa0305bc955
  Stored in directory: /root/.cache/pip/wheels/bf/dd/f9/7084a29342f2fe5fb3c48f35f333f0b106f418999ae63a9890
Successfully built pypianoroll


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from pypianoroll import Multitrack, Track
import glob

def main():
  c=0
  for filename in (glob.glob('drive/My Drive/Synth pop MIDI/*.mid')):
      mtrack = Multitrack(filename)
      mtrack.to_pretty_midi()
      #mtrack = remove_short_tracks(mtrack)
      orchestra = group_by_program(mtrack)
      l=len(mtrack.tracks)
      trackid=[x for x in range(l)]
      newmtrack, inst = merge_instrument(mtrack,orchestra,trackid)
      filename=filename[30:]
      if len(newmtrack.tracks)==4:
        c+=1
        print(filename, len(newmtrack.tracks), inst, orchestra)
        newmtrack.write('drive/My Drive/Synth pop MIDI/four tracks with drum sum/'+filename)
  print(c)

def remove_short_tracks(mtrack):
  l=len(mtrack.tracks)
  i=j=l-1
  al=[]
  while j>=0:
      al.append((mtrack.tracks[j]).get_active_length())
      j-=1
  al.sort()
  a=[]
  while i>=0:
      actl=(mtrack.tracks[i]).get_active_length()
      apr=mtrack.tracks[i].get_active_pitch_range()
      if actl<al[l-1]*0.1 or apr[1]<50:
          a.append(i)
      i-=1
  mtrack.remove_tracks(a)
  return mtrack
    

def group_by_program(mtrack):
  orchestra=[[]for _ in range(5)]
  trackcount=0
  for track in mtrack.tracks:
    if track.program<90:
      if not track.is_drum:
        i=track.program // 25
        orchestra[i].append(trackcount)
      else:
        orchestra[4].append(trackcount)
    trackcount+=1
  orchestra[2]=orchestra[2]+orchestra[3]
  orchestra.remove(orchestra[3])
  return orchestra

def merge_instrument(mtrack,orchestra,trackid):
  #instname=['piano','chromatic percussion','organ','guitar','bass','strings','ensemble','brass','reed','pipe','synth lead','synth pad','synth effects','ethnic','percussive','sound effects']
  instname=['piano','guitar','synth lead','drum']
  drums=[False,False,False,True]
  programs=[5,28,81,0]
  inst=[]
  for p in range(0,4):
    if orchestra[p]!=[]:
      inst.append(instname[p])
      mtrack.merge_tracks(track_indices=orchestra[p], mode='max', program=programs[p], is_drum=drums[p], name=instname[p], remove_merged=False)
  mtrack.remove_tracks(trackid)
  return mtrack, inst

if __name__ == "__main__":
    main()

any.mid 4 ['piano', 'guitar', 'synth lead', 'drum'] [[0, 3, 11], [4], [7], [8]]
higher.mid 4 ['piano', 'guitar', 'synth lead', 'drum'] [[3, 5], [1, 2], [0, 4, 6], [10, 11, 12]]
personal.mid 4 ['piano', 'guitar', 'synth lead', 'drum'] [[2, 3, 5], [0, 6, 8], [7], [4]]
world.mid 4 ['piano', 'guitar', 'synth lead', 'drum'] [[5], [6, 7, 10, 11], [0, 1, 4, 12], [9]]
sweetest.mid 4 ['piano', 'guitar', 'synth lead', 'drum'] [[15, 16], [1, 2, 4, 6, 13, 14], [3, 7, 10, 5, 12], [0, 9]]
inyourroom.mid 4 ['piano', 'guitar', 'synth lead', 'drum'] [[0, 1], [2, 8, 10, 11], [4, 6], [7]]
halo.mid 4 ['piano', 'guitar', 'synth lead', 'drum'] [[2, 4], [0, 10], [3, 6, 1], [7, 8, 9]]
havehold.mid 4 ['piano', 'guitar', 'synth lead', 'drum'] [[2], [0], [4, 6], [5]]
route66.mid 4 ['piano', 'guitar', 'synth lead', 'drum'] [[3, 4, 6], [0, 1], [5], [7]]
neverlet.mid 4 ['piano', 'guitar', 'synth lead', 'drum'] [[6, 8], [1, 2, 4, 9, 10], [0, 7, 11], [3]]
agentorange.mid 4 ['piano', 'guitar', 'synth lead', 'drum'] [[